# Crawling RDF for DBPEDIA Objects in links


## Global variable 

In [67]:
### Global variables 
path="data/" # path where to find data
namesa=['sBT','sprefLabel','sURI','oURI','oprefLabel','oBT', 'KindOfLink'] #column names for training data frame

## Reading a specific validated linkset (Thist2DBPEDIA_ok.csv)

In [68]:
import pandas as pd


# reading Thist2DBPEDIA_ok.csv
THIST2DBPEDIA_df=pd.read_csv(path+'Thist2DBPEDIA_ok.csv',delimiter=";",names=namesa)
#deleting row 0 which contains the old colums name
THIST2DBPEDIA_df=THIST2DBPEDIA_df.drop([0])
THIST2DBPEDIA_df.head()

THIST2DBPEDIA_df.count()


sBT           2304
sprefLabel    3455
sURI          3455
oURI          3455
oprefLabel    3455
oBT           3427
KindOfLink    3428
dtype: int64

In [69]:
import pandas as pd

#setting variable for crawling
pathLDSpider='/Users/bubu/Work/Programming'
pathLDSpider+='/ldspider-1.1e.jar'

pathcrawling=path+'crawling/'
CrawlingDepth="1";
CrawlingNumberOfURI="-1" #not limited

crawledFile=pathcrawling+'DBPEDIA_Crawling'
pathSeed=pathcrawling+'DBPEDIA_Crawling_seeds.csv'
pathLog=pathcrawling+'DBPEDIA_Crawling_seeds.log'

OScrawledFile=pathcrawling+'DBpediaOneSeed'
OSpathSeed=pathcrawling+'DBpediaOneSeed.txt'
OSpathLog=pathcrawling+'DBpediaOneSeed.log'



#Preparing Seed Files

r=THIST2DBPEDIA_df["oURI"].drop_duplicates()
r.to_csv(pathSeed, header=False, index=False, sep='\t', mode='a')



## TEsting crawling on a link


In [70]:
%%bash -s "$OScrawledFile" "$pathLDSpider" "$OSpathSeed" "$OSpathLog" "$CrawlingDepth" "$CrawlingNumberOfURI" 
#echo "$1 $2 $3 $4 $5 $6"
if test -f $1;  then 
 echo "File $1 ; exist";
else
 echo "File $1.nq Not exist; start crawling java -jar $2 -f http://www.w3.org/2004/02/skos/core#broader -f http://www.w3.org/2004/02/skos/core#related -f http://www.w3.org/2004/02/skos/core#narrower -s $3 -a $4  -b $5 $6 -o $1"
 java -jar $2 -s $3 -f http://www.w3.org/2004/02/skos/core#broader -f http://www.w3.org/2004/02/skos/core#related  -f http://www.w3.org/2004/02/skos/core#narrower -a $4 -b $5 $6 -o $1.nq
fi 

File data/crawling/DBpediaOneSeed.nq Not exist; start crawling java -jar /Users/bubu/Work/Programming/ldspider-1.1e.jar -f http://www.w3.org/2004/02/skos/core#broader -f http://www.w3.org/2004/02/skos/core#related -f http://www.w3.org/2004/02/skos/core#narrower -s data/crawling/DBpediaOneSeed.txt -a data/crawling/DBpediaOneSeed.log  -b 1 -1 -o data/crawling/DBpediaOneSeed


May 28, 2019 5:20:01 PM com.ontologycentral.ldspider.Main readSeeds
INFO: read 1 lines from seed file
May 28, 2019 5:20:01 PM com.ontologycentral.ldspider.Main run
INFO: no of seed uris 1
May 28, 2019 5:20:01 PM com.ontologycentral.ldspider.Main run
INFO: frontier done
May 28, 2019 5:20:01 PM com.ontologycentral.ldspider.hooks.links.LinkFilterSelect <init>
INFO: link predicate is [http://www.w3.org/2004/02/skos/core#broader, http://www.w3.org/2004/02/skos/core#related, http://www.w3.org/2004/02/skos/core#narrower]
May 28, 2019 5:20:01 PM com.ontologycentral.ldspider.Main run
INFO: init crawler
May 28, 2019 5:20:01 PM com.ontologycentral.ldspider.http.internal.CloseIdleConnectionThread <init>
INFO: Initialised CloseIdleConnectionThread with sleepTime 60000 ms
May 28, 2019 5:20:01 PM com.ontologycentral.ldspider.http.internal.CloseIdleConnectionThread run
INFO: Starting CloseIdleConnectionThread
May 28, 2019 5:20:01 PM com.ontologycentral.ldspider.http.internal.CloseIdleConnectionThread 

## Let's convert a NQuad into a NTriple

In [71]:
%%bash -s "$path" "$OScrawledFile" "$pathcrawling"
java arq.sparql  --data $2.nq --query $3/NqToNt.sparql > $2.ttl


In [72]:
%%bash -s "$path" "$OScrawledFile" "$pathcrawling"
java arq.sparql  --data $2.ttl \
 --query $3/all.sparql --results CSV  > $2.csv


## A bash for crawling via LDspider

In [78]:
%%bash -s "$crawledFile" "$pathLDSpider" "$pathSeed" "$pathLog" "$CrawlingDepth" "$CrawlingNumberOfURI" 
#echo "$1 $2 $3 $4 $5 $6"
if test -f $1;  then 
 echo "File $1 ; exist";
else
 echo "File $1 Not exist; start crawling java -jar $2 -s $3 -f http://www.w3.org/2004/02/skos/core#broader -f http://www.w3.org/2004/02/skos/core#related  -f http://www.w3.org/2004/02/skos/core#narrower  -a $4 -b $5 $6 -o $1.nq"
 java -jar $2 -s $3 -f http://www.w3.org/2004/02/skos/core#broader -f http://www.w3.org/2004/02/skos/core#related -f http://www.w3.org/2004/02/skos/core#narrower -a $4  -b $5 $6 -o $1.nq
fi 

File data/crawling/DBPEDIA_Crawling Not exist; start crawling java -jar /Users/bubu/Work/Programming/ldspider-1.1e.jar -s data/crawling/DBPEDIA_Crawling_seeds.csv -f http://www.w3.org/2004/02/skos/core#broader -f http://www.w3.org/2004/02/skos/core#related  -f http://www.w3.org/2004/02/skos/core#narrower  -a data/crawling/DBPEDIA_Crawling_seeds.log -b 1 -1 -o data/crawling/DBPEDIA_Crawling.nq


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [80]:
%%bash -s "$path" "$crawledFile" "$pathcrawling"
java  -Xmx6072M  arq.sparql  --data $2.nq --query $3/NqToNt.sparql > $2.ttl